# Chapter 3: Attention Mechanisms

In [1]:
from importlib.metadata import version

print("torch version:", version("torch"))

torch version: 2.9.0


`inputs`:  $x^{(i)}_{a}$

$i$ is the sequence index and $a$ is the feature index

In [2]:
import torch

inputs = torch.tensor(
  [[0.43, 0.15, 0.89], # Your     (x^1)
   [0.55, 0.87, 0.66], # journey  (x^2)
   [0.57, 0.85, 0.64], # starts   (x^3)
   [0.22, 0.58, 0.33], # with     (x^4)
   [0.77, 0.25, 0.10], # one      (x^5)
   [0.05, 0.80, 0.55]] # step     (x^6)
)

`query`: $q_{a} = x^{(2)}_{a}$

`attn_scores_2`: $\omega_{2i} = \sum_{a} q_{a} x^{(i)}_{a}$

In [3]:
query = inputs[1] #x(2) in book
attn_scores_2 = torch.empty(inputs.shape[0])
for i,x in enumerate(inputs):
    attn_scores_2[i] = torch.dot(inputs[i],query)
print('un-normalized scores:',attn_scores_2)

attn_scores_2_tmp = attn_scores_2 / attn_scores_2.sum()
print('normalized scores:', attn_scores_2_tmp)

un-normalized scores: tensor([0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865])
normalized scores: tensor([0.1455, 0.2278, 0.2249, 0.1285, 0.1077, 0.1656])


Softmax implementation
Input is `x` which is expected to be the attention scores $\omega_{2i}$, here is we are using element 2 as the query, this is a 1-D vector in the sequence index.

Attention weights `attn_weights_2` : $\alpha_{2i} = \text{softmax}(\omega_{2i})$

In [4]:
def softmax_naive(x):
    return torch.exp(x) / torch.exp(x).sum()
attn_weights_2= softmax_naive(attn_scores_2)
print(attn_weights_2)

tensor([0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581])


In [5]:
attn_weights_2 = torch.softmax(attn_scores_2, dim=0)
print(attn_weights_2)

tensor([0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581])


`context_vec_2`: $z^{(2)}_{a} = \sum_i \alpha_{2i} x^{(i)}_{a}$

In [6]:
context_vec_2 = torch.zeros(query.shape)
for i,x_i in enumerate(inputs):
    context_vec_2 += attn_weights_2[i]*x_i
print(context_vec_2)

tensor([0.4419, 0.6515, 0.5683])


### 3.3.2 Generalizing to all input sequence tokens
`attn_scores` = $\omega_{ij} = \sum_{a} x^{(i)}_{a}x^{(j)}_{a}$, which is an $n \times n$ tensor, where $n$ is the sequence index.

In [7]:
n_seq = inputs.shape[0]
attn_scores = torch.empty(n_seq,n_seq)

for i,x_i in enumerate(inputs):
    for j, x_j in enumerate(inputs):
        attn_scores[i][j] = torch.dot(x_i,x_j)
print(attn_scores)

tensor([[0.9995, 0.9544, 0.9422, 0.4753, 0.4576, 0.6310],
        [0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865],
        [0.9422, 1.4754, 1.4570, 0.8296, 0.7154, 1.0605],
        [0.4753, 0.8434, 0.8296, 0.4937, 0.3474, 0.6565],
        [0.4576, 0.7070, 0.7154, 0.3474, 0.6654, 0.2935],
        [0.6310, 1.0865, 1.0605, 0.6565, 0.2935, 0.9450]])


In [8]:
attn_scores = inputs @ inputs.T # 6x3 . (6x3)^T = 6x3 . 3x6
print(attn_scores)

tensor([[0.9995, 0.9544, 0.9422, 0.4753, 0.4576, 0.6310],
        [0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865],
        [0.9422, 1.4754, 1.4570, 0.8296, 0.7154, 1.0605],
        [0.4753, 0.8434, 0.8296, 0.4937, 0.3474, 0.6565],
        [0.4576, 0.7070, 0.7154, 0.3474, 0.6654, 0.2935],
        [0.6310, 1.0865, 1.0605, 0.6565, 0.2935, 0.9450]])


`attn_weights` = $\alpha_{ij} = \text{softmax}_{j}(\omega_{ij})$ 

In [9]:
attn_weights = torch.softmax(attn_scores,dim=-1)
print(attn_weights)

tensor([[0.2098, 0.2006, 0.1981, 0.1242, 0.1220, 0.1452],
        [0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581],
        [0.1390, 0.2369, 0.2326, 0.1242, 0.1108, 0.1565],
        [0.1435, 0.2074, 0.2046, 0.1462, 0.1263, 0.1720],
        [0.1526, 0.1958, 0.1975, 0.1367, 0.1879, 0.1295],
        [0.1385, 0.2184, 0.2128, 0.1420, 0.0988, 0.1896]])


In [10]:
## check that the rows sum to 1
attn_weights.sum(dim=-1)

tensor([1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000])

`all_context_vecs` = $z^{i}_{a} = \sum_{j} x^{j}_{a} \alpha_{ij}$

(n,d) = (n,n) x (n,d)

In [11]:
all_context_vecs = attn_weights @ inputs
print(all_context_vecs)

tensor([[0.4421, 0.5931, 0.5790],
        [0.4419, 0.6515, 0.5683],
        [0.4431, 0.6496, 0.5671],
        [0.4304, 0.6298, 0.5510],
        [0.4671, 0.5910, 0.5266],
        [0.4177, 0.6503, 0.5645]])


In [12]:
print(all_context_vecs[1]) #should match what we had before
print(context_vec_2)

tensor([0.4419, 0.6515, 0.5683])
tensor([0.4419, 0.6515, 0.5683])


## 3.4 Trainable weights

In this example the model dimension is `d_in` (3) but the Q,K,V vectors live in a
different, lower dimensional space `d_out` (2). Presumably this makes it a little easier to generalize to multiple heads where `d_out` typically is `d_in / n_heads`

In [13]:
x_2 = inputs[1]
d_in = inputs.shape[1]
d_out = 2
torch.manual_seed(123)
W_q = torch.nn.Parameter(torch.rand(d_in, d_out), requires_grad=False)
W_k = torch.nn.Parameter(torch.rand(d_in, d_out), requires_grad=False)
W_v = torch.nn.Parameter(torch.rand(d_in, d_out), requires_grad=False)

q_2 = x_2 @ W_q # (3) x (3x2) -> 2
k_2 = x_2 @ W_k
v_2 = x_2 @ W_v

print(q_2)

tensor([0.4306, 1.4551])


In [14]:
keys = inputs @ W_k
values = inputs @ W_v
queries = inputs @ W_v

In [15]:
#omega_{22}
keys_2 = keys[1]
attn_score_22 = q_2.dot(k_2)
print(attn_score_22)

tensor(1.8524)


`attn_scores_2` = $\omega_{2j} = \sum_{a} q^{(2)}_{a} k^{(j)}_{a}$

In [16]:
attn_scores_2 = q_2 @ keys.T
print('omega_{2j}: ',attn_scores_2)

omega_{2j}:  tensor([1.2705, 1.8524, 1.8111, 1.0795, 0.5577, 1.5440])


`attn_weights_2` = $\alpha_{2j} = \text{softmax}(\omega_{2j}/\sqrt{d_{out}},j)$

In [17]:
attn_weights_2 = torch.softmax(attn_scores_2/d_out**0.5, dim=-1)
print(attn_weights_2)

tensor([0.1500, 0.2264, 0.2199, 0.1311, 0.0906, 0.1820])


`context_vec_2` = $z^{(2)}_{a} = \sum_{j}  \alpha_{2j} v^{(j)}_{a}$

In [19]:
context_vec_2 = attn_weights_2 @ values
print(context_vec_2)

tensor([0.3061, 0.8210])


#### Self attention class
`class SelfAttention_v1`

Subclass `nn.Module`. Implement `__init__`, and `forward(self,x)`

In [31]:
import torch.nn as nn

class SelfAttention_v1(nn.Module):
    def __init__(self, d_in, d_out):
        super().__init__()
        #define the weights
        self.W_q = torch.nn.Parameter(torch.rand(d_in, d_out))
        self.W_k = torch.nn.Parameter(torch.rand(d_in, d_out))
        self.W_v = torch.nn.Parameter(torch.rand(d_in, d_out))

    def forward(self, x): # x: shape(...,n,d_in)

        #shape(...,n,d_in) x (d_in,d_out) -> shape(...,n,d_out)
        queries = x @ self.W_q 
        keys = x @ self.W_k
        values = x @ self.W_v
        

        #(n,d_out) x (n,d_out)^T -> shape(n,n)
        omega = queries @ keys.T
        alpha = torch.softmax(omega/d_out**0.5,dim=-1)
        context_vec = alpha @ values
        return context_vec

In [32]:
#now test it
torch.manual_seed(123)
SA_v1 = SelfAttention_v1(d_in,d_out)
SA_v1.forward(inputs)

tensor([[0.2996, 0.8053],
        [0.3061, 0.8210],
        [0.3058, 0.8203],
        [0.2948, 0.7939],
        [0.2927, 0.7891],
        [0.2990, 0.8040]], grad_fn=<MmBackward0>)

#### New self attention class using linear layers for holding parameters
`class SelfAttention_v2`

Subclass `nn.Module`. Implement `__init__`, and `forward(self,x)`

In [34]:
class SelfAttention_v2(nn.Module):
    def __init__(self, d_in, d_out, qkv_bias=False):
        super().__init__()
        #define the weights
        self.W_q = nn.Linear(d_in, d_out,bias=qkv_bias)
        self.W_k = nn.Linear(d_in, d_out,bias=qkv_bias)
        self.W_v = nn.Linear(d_in, d_out,bias=qkv_bias)

    def forward(self, x): # x: shape(...,n,d_in)

        #shape(...,n,d_in) x (d_in,d_out) -> shape(...,n,d_out)
        queries = self.W_q(x) 
        keys = self.W_k(x)
        values = self.W_v(x)
        

        #(n,d_out) x (n,d_out)^T -> shape(n,n)
        omega = queries @ keys.T
        alpha = torch.softmax(omega/d_out**0.5,dim=-1)
        context_vec = alpha @ values
        return context_vec
    
torch.manual_seed(789)
SA_v2 = SelfAttention_v2(d_in,d_out)
SA_v2.forward(inputs)

tensor([[-0.0739,  0.0713],
        [-0.0748,  0.0703],
        [-0.0749,  0.0702],
        [-0.0760,  0.0685],
        [-0.0763,  0.0679],
        [-0.0754,  0.0693]], grad_fn=<MmBackward0>)

## 3.5 Causal attention and masking